<a href="https://colab.research.google.com/github/kcast3/Simulaci-n/blob/main/Movimiento_de_proyectiles_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# --- Celda 1: Instalación de Librerías ---
# Necesitamos ipywidgets para los deslizadores interactivos.
# Aseguramos que todas las librerías necesarias estén instaladas.
!pip install numpy matplotlib ipywidgets
print("Librerías instaladas y/o actualizadas.")

import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, Markdown, Math
import ipywidgets as widgets
from ipywidgets import interact, fixed

# Se puede ajustar el límite de incrustación de animaciones si es necesario,
# aunque para gráficas estáticas interactivas no es estrictamente necesario.
# plt.rcParams['animation.embed_limit'] = 2**128
print("Librerías importadas correctamente.")


# --- Celda 2: Definición de Parámetros y Ecuaciones ---

g = 9.81  # Aceleración debido a la gravedad (m/s^2)

def calcular_posicion(v0, theta, h0, t):
    """
    Calcula la posición (x, y) de un proyectil en un tiempo 't' con altura inicial.
    v0: Velocidad inicial (m/s)
    theta: Ángulo de lanzamiento (radianes)
    h0: Altura inicial (m)
    t: Tiempo (s)
    """
    x = v0 * np.cos(theta) * t
    y = h0 + v0 * np.sin(theta) * t - 0.5 * g * t**2
    return x, y

def calcular_velocidad(v0, theta, t):
    """
    Calcula la velocidad (vx, vy) de un proyectil en un tiempo 't'.
    """
    vx = np.full_like(t, v0 * np.cos(theta)) # v_x es constante, se usa np.full_like para generar un array del mismo tamaño que t
    vy = v0 * np.sin(theta) - g * t
    return vx, vy

def calcular_aceleracion():
    """
    La aceleración en el movimiento de proyectiles es constante (ax=0, ay=-g).
    """
    ax = 0
    ay = -g
    return ax, ay

def calcular_tiempo_vuelo_con_altura(v0, theta, h0):
    """
    Calcula el tiempo total de vuelo del proyectil, considerando una altura inicial h0.
    Resuelve la ecuación cuadrática y = h0 + v0*sin(theta)*t - 0.5*g*t^2 = 0
    """
    a = -0.5 * g
    b = v0 * np.sin(theta)
    c = h0
    discriminante = b**2 - 4 * a * c

    if discriminante < 0:
        # Si el discriminante es negativo, no hay soluciones reales (el proyectil no llega a y=0).
        # Esto puede ocurrir si el proyectil ya está por debajo de y=0 y se lanza hacia abajo,
        # o si la trayectoria nunca cruza y=0.
        return 0

    t1 = (-b + np.sqrt(discriminante)) / (2 * a)
    t2 = (-b - np.sqrt(discriminante)) / (2 * a)

    # El tiempo de vuelo es el valor positivo más grande que sea relevante.
    return max(t1, t2, 0)


def calcular_alcance_maximo_con_altura(v0, theta, h0):
    """
    Calcula el alcance horizontal máximo del proyectil con altura inicial.
    """
    tiempo_vuelo = calcular_tiempo_vuelo_con_altura(v0, theta, h0)
    alcance = v0 * np.cos(theta) * tiempo_vuelo
    return alcance

def calcular_altura_maxima(v0, theta, h0):
    """
    Calcula la altura máxima alcanzada por el proyectil, considerando altura inicial.
    """
    t_altura_max = (v0 * np.sin(theta)) / g

    # Si el tiempo para la altura máxima es negativo (lo que significa que el proyectil
    # fue lanzado hacia abajo desde el inicio), o si la componente vertical de la velocidad
    # inicial es negativa, la altura máxima es la altura inicial.
    if t_altura_max < 0 or (v0 * np.sin(theta) < 0):
        return h0

    return h0 + (v0**2 * np.sin(theta)**2) / (2 * g)

print("Funciones de cálculo de movimiento de proyectiles definidas.")

# --- Celda 3: Función Principal de Simulación y Graficación (modificada para interactividad) ---

def simular_proyectil_interactivo(v0, angulo_grados, h0):
    """
    Realiza la simulación y grafica el movimiento del proyectil,
    mostrando las ecuaciones relevantes. Esta función está diseñada para interactuar con ipywidgets.
    v0: Velocidad inicial (m/s)
    angulo_grados: Ángulo de lanzamiento (grados)
    h0: Altura inicial (m)
    """
    theta = np.deg2rad(angulo_grados)  # Convertir grados a radianes

    tiempo_vuelo = calcular_tiempo_vuelo_con_altura(v0, theta, h0)

    if tiempo_vuelo <= 0:
        # Si el tiempo de vuelo es cero o negativo, creamos un rango de tiempo pequeño
        # para que las gráficas no queden vacías.
        t = np.linspace(0, 0.1, 2)
    else:
        t = np.linspace(0, tiempo_vuelo, 500)

    x, y = calcular_posicion(v0, theta, h0, t)
    vx, vy = calcular_velocidad(v0, theta, t)
    ax, ay = calcular_aceleracion()

    # --- Gráficas ---
    # Cerramos cualquier figura anterior para evitar superposiciones al actualizar.
    plt.close('all')
    fig, axs = plt.subplots(3, 1, figsize=(10, 18)) # Ajustar figsize para más espacio

    # Gráfica de Posición (Trayectoria)
    axs[0].plot(x, y, 'b-', label='Trayectoria')
    axs[0].plot(x[0], y[0], 'go', markersize=8, label='Inicio')
    # Solo marcamos el punto final si el tiempo de vuelo es significativo
    if tiempo_vuelo > 0.01:
        axs[0].plot(x[-1], y[-1], 'ro', markersize=8, label='Fin (aterrizaje)')

    axs[0].set_xlabel('Posición Horizontal (m)')
    axs[0].set_ylabel('Posición Vertical (m)')
    axs[0].set_title('Trayectoria del Proyectil')
    axs[0].grid(True)
    axs[0].axhline(0, color='black', linewidth=0.5) # Línea en y=0
    axs[0].axvline(0, color='black', linewidth=0.5) # Línea en x=0

    # Ajuste dinámico de los límites de los ejes para que la trayectoria se vea bien
    x_max_plot = max(x) * 1.1 if len(x) > 0 and max(x) > 0 else 10 # Asegura un límite mínimo si x es 0 o negativo
    y_max_plot = max(y) * 1.1 if len(y) > 0 and max(y) > h0 else h0 + 5 # Asegura un límite mínimo si y es solo h0
    y_min_plot = min(y) * 1.1 if len(y) > 0 and min(y) < 0 else 0 # Permite ver caídas por debajo de 0

    axs[0].set_xlim(0, x_max_plot)
    axs[0].set_ylim(y_min_plot, y_max_plot)

    axs[0].legend()
    axs[0].set_aspect('equal', adjustable='box') # Mantiene la proporción para una trayectoria realista


    # Mostrar ecuaciones de posición directamente debajo de la gráfica
    display(Markdown("### Ecuaciones de Posición"))
    display(Math(r'x(t) = (v_0 \cos \theta) t'))
    display(Math(r'y(t) = h_0 + (v_0 \sin \theta) t - \frac{1}{2} g t^2'))

    # Gráfica de Velocidad
    axs[1].plot(t, vx, 'r--', label='Velocidad Horizontal ($v_x$)')
    axs[1].plot(t, vy, 'b--', label='Velocidad Vertical ($v_y$)')
    axs[1].set_xlabel('Tiempo (s)')
    axs[1].set_ylabel('Velocidad (m/s)')
    axs[1].set_title('Componentes de la Velocidad vs. Tiempo')
    axs[1].grid(True)
    axs[1].legend()

    # Mostrar ecuaciones de velocidad
    display(Markdown("### Ecuaciones de Velocidad"))
    display(Math(r'v_x(t) = v_0 \cos \theta \quad \text{(constante)}'))
    display(Math(r'v_y(t) = v_0 \sin \theta - g t'))

    # Gráfica de Aceleración
    # Usamos [ax]*len(t) y [ay]*len(t) para crear arrays del mismo tamaño que t
    # ya que plot() espera arrays.
    axs[2].plot(t, [ax]*len(t), 'g:', label='Aceleración Horizontal ($a_x = 0$)')
    axs[2].plot(t, [ay]*len(t), 'm:', label='Aceleración Vertical ($a_y = -g$)')
    axs[2].set_xlabel('Tiempo (s)')
    axs[2].set_ylabel('Aceleración (m/s^2)')
    axs[2].set_title('Componentes de la Aceleración vs. Tiempo')
    axs[2].grid(True)
    axs[2].legend()
    # Ajustamos los límites del eje y para la aceleración para que sea visible
    axs[2].set_ylim(min(ay, -1) * 1.2, max(ax, 1) * 1.2)


    # Mostrar ecuaciones de aceleración
    display(Markdown("### Ecuaciones de Aceleración"))
    display(Math(r'a_x = 0'))
    display(Math(r'a_y = -g'))

    # Ajusta automáticamente los parámetros de la subtrama para que quepan
    # sin superponerse a los títulos de las figuras o a los objetos x y y.
    plt.tight_layout(rect=[0, 0.03, 1, 0.96])
    plt.show() # Muestra las gráficas

    # --- Resumen de Ecuaciones y Cálculos Clave ---
    print("\n--- Resumen de Ecuaciones y Cálculos Clave ---")
    display(Markdown(f"**Condiciones Iniciales:**"))
    display(Math(f'v_0 = {v0:.2f}\\, \\text{{m/s}}'))
    display(Math(f'\\theta = {angulo_grados:.2f}° \\approx {theta:.2f}\\, \\text{{rad}}'))
    display(Math(f'h_0 = {h0:.2f}\\, \\text{{m}}'))
    display(Math(f'g = {g:.2f}\\, \\text{{m/s}}^2'))

    display(Markdown(f"\n**Tiempo de Vuelo ($T$)**"))
    display(Math(r'\text{Se calcula resolviendo: } h_0 + (v_0 \sin \theta) T - \frac{1}{2} g T^2 = 0'))
    display(Markdown(f"**Resultado:** $T = {calcular_tiempo_vuelo_con_altura(v0, theta, h0):.2f}\\, \\text{{s}}$"))

    display(Markdown(f"\n**Alcance Máximo Horizontal ($R$)**"))
    display(Math(r'R = (v_0 \cos \theta) T'))
    display(Markdown(f"**Resultado:** $R = {calcular_alcance_maximo_con_altura(v0, theta, h0):.2f}\\, \\text{{m}}$"))

    display(Markdown(f"\n**Altura Máxima ($H_{max}$)**"))
    display(Math(r'H_{max} = h_0 + \frac{(v_0 \sin \theta)^2}{2g} \quad \text{(si } v_0 \sin \theta \ge 0 \text{)}'))
    display(Math(r'\text{Si } v_0 \sin \theta < 0 \text{ (se lanza hacia abajo), } H_{max} = h_0'))
    display(Markdown(f"**Resultado:** $H_{max} = {calcular_altura_maxima(v0, theta, h0):.2f}\\, \\text{{m}}$"))

print("Función 'simular_proyectil_interactivo' definida y lista para usar.")

# --- Celda 4: Ejecución de la Simulación Interactiva ---

# Mensaje de Bienvenida y Explicación
display(Markdown("""
---
### ¡Bienvenido al Simulador Interactivo de Proyectiles!

Este programa te permite explorar de forma dinámica el movimiento de un proyectil. Podrás ajustar la **velocidad inicial**, el **ángulo de lanzamiento** y la **altura inicial** usando los deslizadores. Cada vez que muevas un deslizador, las gráficas de la trayectoria, velocidad y aceleración se actualizarán automáticamente, junto con los cálculos clave como el tiempo de vuelo, el alcance y la altura máxima. ¡Experimenta y observa cómo cambian los resultados con cada ajuste!
---
"""))

print("Mueve los deslizadores para actualizar las gráficas y los cálculos en tiempo real:")

interact(simular_proyectil_interactivo,
         v0=widgets.FloatSlider(min=1, max=100, step=1, value=35, description='Velocidad Inicial (m/s)'),
         angulo_grados=widgets.IntSlider(min=0, max=90, step=1, value=45, description='Ángulo (grados)'),
         h0=widgets.FloatSlider(min=0, max=100, step=1, value=0, description='Altura Inicial (m)'));

Librerías instaladas y/o actualizadas.
Librerías importadas correctamente.
Funciones de cálculo de movimiento de proyectiles definidas.
Función 'simular_proyectil_interactivo' definida y lista para usar.



---
### ¡Bienvenido al Simulador Interactivo de Proyectiles!

Este programa te permite explorar de forma dinámica el movimiento de un proyectil. Podrás ajustar la **velocidad inicial**, el **ángulo de lanzamiento** y la **altura inicial** usando los deslizadores. Cada vez que muevas un deslizador, las gráficas de la trayectoria, velocidad y aceleración se actualizarán automáticamente, junto con los cálculos clave como el tiempo de vuelo, el alcance y la altura máxima. ¡Experimenta y observa cómo cambian los resultados con cada ajuste!
---


Mueve los deslizadores para actualizar las gráficas y los cálculos en tiempo real:


interactive(children=(FloatSlider(value=35.0, description='Velocidad Inicial (m/s)', min=1.0, step=1.0), IntSl…